In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/gorani/gorani/backend/dataserver


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('Classify Readable Book')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters
auwpp_threshold = 0.3
wps_threshold = 67 / 60

In [5]:
from gorani.spark import read_data_all
import pyspark.sql.functions as F

time_df = read_data_all(spark, 'book_estimated_time')\
        .select('user_id', 'book_id', (F.col('total_words') / F.col('estimated_time')).alias('wps'))
time_df.show()

auwpp_df = read_data_all(spark, 'user_auwpps')
auwpp_df.show()

+-------+-------+------------------+
|user_id|book_id|               wps|
+-------+-------+------------------+
|     10|      1|56.608099594760624|
|     10|      2| 57.48199949651947|
|     10|      3|  57.0865967215464|
|     10|      4| 57.36063352930009|
|     10|      5|55.190594395995134|
|     10|      6| 56.98904088357437|
|      1|      1|145.57408325858702|
|      1|      2|126.84137269227143|
|      1|      3|112.64418121025594|
|      1|      4|121.29961552563137|
|      1|      5|224.48510657995232|
|      1|      6|136.87691641313896|
+-------+-------+------------------+

+-------+-------+----------+
|user_id|book_id|     auwpp|
+-------+-------+----------+
|     10|      1| 0.8691228|
|     10|      2| 0.8658228|
|     10|      3|0.86518097|
|     10|      4|0.87211883|
|     10|      5| 0.8648834|
|     10|      6|     0.867|
|      1|      1| 0.9767935|
|      1|      2| 0.9707848|
|      1|      3|0.97398096|
|      1|      4|0.97128713|
|      1|      5| 0.9704294|
|

In [6]:
from gorani.spark import write_data
result_df = time_df.join(auwpp_df.alias('f'), (time_df['user_id'] == auwpp_df['user_id']) & (time_df['book_id'] == auwpp_df['book_id']), 'inner')\
        .drop(F.col('f.user_id'))\
        .drop(F.col('f.book_id'))\
        .where((F.col('auwpp') <= auwpp_threshold) | (F.col('wps') >= wps_threshold))\
        .drop(F.col('wps'))\
        .drop(F.col('auwpp'))

result_df.show()

write_data('readable_books', result_df)

print('success')

+-------+-------+
|user_id|book_id|
+-------+-------+
|     10|      1|
|     10|      2|
|     10|      3|
|     10|      4|
|     10|      5|
|     10|      6|
|      1|      1|
|      1|      2|
|      1|      3|
|      1|      4|
|      1|      5|
|      1|      6|
+-------+-------+

success
